# Capstone Project

## Park Slope Parents Membership
#### Part 2c: Time Series

In [83]:
# Load libraries

from datetime import datetime, date, timedelta
import csv
import pandas as pd 
import plotly.graph_objs as go
import plotly.plotly as py
py.sign_in('ajbentley', 'zjfu2vasav')



# import matplotlib.pyplot as plt
# %matplotlib inline

# from IPython.display import HTML
# from matplotlib import cm as cm
# from mpl_toolkits.mplot3d import Axes3D
# from pandas.io import sql
# from sklearn import metrics
# from sklearn.cluster import DBSCAN
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA 
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.metrics import silhouette_score
# from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
# from sqlalchemy import create_engine
# import calendar

# import numpy as np

# import patsy

# from plotly.tools import FigureFactory as FF
# from plotly.graph_objs import graph_objs
# import psycopg2 as psy
# import scipy
# import seaborn as sns
# import sklearn


In [84]:
# read in data

dfn = pd.read_csv("../../projects/psp/refined_data/psp_numerical.csv")
dfn.info()
# dfn.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14591 entries, 0 to 14590
Data columns (total 66 columns):
mem_no                                                                            14591 non-null object
city                                                                              14591 non-null object
state                                                                             14591 non-null object
zip                                                                               14591 non-null int64
joined                                                                            14591 non-null object
exp_date                                                                          14591 non-null object
status                                                                            14591 non-null object
mem_type                                                                          14591 non-null int64
last_renewal_date                                            

In [85]:
dfn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14591 entries, 0 to 14590
Data columns (total 66 columns):
mem_no                                                                            14591 non-null object
city                                                                              14591 non-null object
state                                                                             14591 non-null object
zip                                                                               14591 non-null int64
joined                                                                            14591 non-null object
exp_date                                                                          14591 non-null object
status                                                                            14591 non-null object
mem_type                                                                          14591 non-null int64
last_renewal_date                                            

In [86]:
dfn.T.tail(20)

,0,1,2,3,4,5,6,7,8,9,...,14581,14582,14583,14584,14585,14586,14587,14588,14589,14590
tony_dids_No,0,1,1,0,1,0,1,0,0,0,...,1,0,1,0,0,1,1,1,1,1
tony_dids_Yes,1,0,0,1,0,1,0,1,1,1,...,0,1,0,1,1,0,0,0,0,0
discovered_A PSP member I don't know told me about it,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
discovered_A PSP member who is a friend/neighbor,0,0,1,1,1,0,1,1,0,1,...,0,1,1,1,0,1,1,0,1,1
discovered_Found it through Yahoo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
discovered_Found it through a Google search,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
"discovered_Heard about it on another online parenting group (Urban Baby, etc.)",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"discovered_Heard about it through a magazine, newspaper, blog",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
discovered_I don't remember,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
discovered_Other,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


I discovered that in 2007 PSP got a website and in 2009 began charging for membership. After exploring year to year data I concluded that these changes were disruptive / significant and chose to cut data prior to 2010.

In order to cut those dates without losing members who joined prior to 2010 I'll first dummy out columns representing membership in each year and will then drop rows that don't indicate membership in 2010 or sooner.

In [87]:
dfn.exp_year.max()

2040

In [88]:
# expyear = pd.Series(dfn.exp_year)
# expyear.drop(([2014, 2012, 2013, 2010, 2011, 2015, 2009]), inplace=True)
# expyear = int([2020, 2017, 2019, 2018, 2016, 2029, 2030, 2040, 2039, 2021, 2034, 2025])
# expyear.unique()


In [89]:
# isolate members by year

annual_mem_cols = ['mem_no','joined', 'exp_date', 'exp_year']
dfy = pd.DataFrame(dfn[annual_mem_cols])

dfy['joined'] = pd.to_datetime(dfy['joined'])
dfy['exp_date'] = pd.to_datetime(dfy['exp_date'])

year_list = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

for year in year_list:
    
    dfy['mem_' + str(year)] = dfy.apply(lambda x: x['joined'].year <= year and x['exp_date'].\
                                        year>= year, axis=1).astype('int')

In [90]:
memlist_2016 = []

for n in dfn.exp_year:
    if n == 2016:
        memlist_2016.append(1)
    else:
        memlist_2016.append(0)

        
m16 = pd.Series(memlist_2016)
        
dfy = pd.concat([dfy, m16], axis = 1)
dfy.rename (columns={0:'mem_2016'}, inplace=True)
dfy.head()

,mem_no,joined,exp_date,exp_year,mem_2002,mem_2003,mem_2004,mem_2005,mem_2006,mem_2007,mem_2008,mem_2009,mem_2010,mem_2011,mem_2012,mem_2013,mem_2014,mem_2015,mem_2016
0,00004,2009-04-13,2020-04-12,2020,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0
1,00101,2002-07-17,2014-09-15,2014,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
2,00118,2002-08-13,2017-06-15,2017,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
3,00121,2002-08-26,2019-06-16,2019,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0
4,00122,2002-08-28,2017-06-20,2017,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0


In [91]:
### Prior attempt
        
# ml16 = pd.Series(memlist_2016)

# def ml16(x,y):
#     for n in x:
#         if n <= 2016:
#             for n in y:
#                 if n>= 2016:
#                     memlist_2016.append(1)
#         else:
#             memlist_2016.append(0)  


# ml16(dfn['join_year'], dfn['exp_year'])            
            
# for row in dfy:
#     if 'join_year' <= 2016 | 'exp_year' >= 2016:
#         memlist_2016.append(1)
#     else:
#         memlist_2016.append(0)



# smem_2016 = pd.Series(memlist_2016)

# dfy['mem_2016'] = smem_2016

# dfy['mem_2016'] = dfy.apply(lambda x: x['exp_date'] <= 2016, axis=1).astype('int')

# dfy.head()

##### previous attempt, which stopped indicating membership after it found a first year

# for year in year_list:

#     mem_year_list = []
    
#     for x in dfy.join_year:
#         while x == year:
#             if x >= year | x < (year+1):
#                 mem_year_list.append(1)
#             else:
#                 mem_year_list.append(0)

#     myl = pd.Series(mem_year_list)
#     dfy[year] = myl
#     er = str(year)
#     er = ('mem_'+ er[2:])
#     dfy.rename (columns={year:er}, inplace=True)

In [92]:
dfy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14591 entries, 0 to 14590
Data columns (total 19 columns):
mem_no      14591 non-null object
joined      14591 non-null datetime64[ns]
exp_date    14591 non-null datetime64[ns]
exp_year    14591 non-null int64
mem_2002    14591 non-null int64
mem_2003    14591 non-null int64
mem_2004    14591 non-null int64
mem_2005    14591 non-null int64
mem_2006    14591 non-null int64
mem_2007    14591 non-null int64
mem_2008    14591 non-null int64
mem_2009    14591 non-null int64
mem_2010    14591 non-null int64
mem_2011    14591 non-null int64
mem_2012    14591 non-null int64
mem_2013    14591 non-null int64
mem_2014    14591 non-null int64
mem_2015    14591 non-null int64
mem_2016    14591 non-null int64
dtypes: datetime64[ns](2), int64(16), object(1)
memory usage: 2.1+ MB


In [93]:
# annual_mem_cols = ['mem_no','joined', 'exp_date']
# dfy = pd.DataFrame(dfn[annual_mem_cols])

# dfy['joined'] = pd.to_datetime(dfy['joined'])
# dfy['exp_date'] = pd.to_datetime(dfy['exp_date'])

# df1=pd.concat([pd.Series(r.Index, 
#                          pd.date_range(r.joined,r.exp_date+pd.offsets.YearEnd(1), freq='A')) 
#                for r in dfy.itertuples()]).reset_index()
# df1.columns=[ 'years', 'index']
# df1.years = df1.years.dt.year
# df1['vals'] = 1
# df1 = df1.pivot(index='index', columns='years', values='vals').fillna(0).astype(int)
# print (df1)

dfy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14591 entries, 0 to 14590
Data columns (total 19 columns):
mem_no      14591 non-null object
joined      14591 non-null datetime64[ns]
exp_date    14591 non-null datetime64[ns]
exp_year    14591 non-null int64
mem_2002    14591 non-null int64
mem_2003    14591 non-null int64
mem_2004    14591 non-null int64
mem_2005    14591 non-null int64
mem_2006    14591 non-null int64
mem_2007    14591 non-null int64
mem_2008    14591 non-null int64
mem_2009    14591 non-null int64
mem_2010    14591 non-null int64
mem_2011    14591 non-null int64
mem_2012    14591 non-null int64
mem_2013    14591 non-null int64
mem_2014    14591 non-null int64
mem_2015    14591 non-null int64
mem_2016    14591 non-null int64
dtypes: datetime64[ns](2), int64(16), object(1)
memory usage: 2.1+ MB


In [94]:
# cut dfy down so there won't be repeated columns when concat with dfn

dfy.drop(['mem_no','joined', 'exp_date', 'exp_year'], axis=1, inplace=True)


dfn = pd.concat([dfn, dfy], axis = 1)


dfn.head()

,mem_no,city,state,zip,joined,exp_date,status,mem_type,last_renewal_date,gender,...,mem_2007,mem_2008,mem_2009,mem_2010,mem_2011,mem_2012,mem_2013,mem_2014,mem_2015,mem_2016
0,00004,brooklyn,NY,11215,2009-04-13,2020-04-12,Active,5,2009-04-13,Female,...,0,0,1,1,1,1,1,1,1,0
1,00101,brooklyn,NY,11215,2002-07-17,2014-09-15,Expired,0,2002-07-17,Female,...,1,1,1,1,1,1,1,1,0,0
2,00118,brooklyn,NY,11215,2002-08-13,2017-06-15,Active,0,2002-08-13,Female,...,1,1,1,1,1,1,1,1,1,0
3,00121,brooklyn,NY,11215,2002-08-26,2019-06-16,Active,2,2002-08-26,Female,...,1,1,1,1,1,1,1,1,1,0
4,00122,brooklyn,NY,11215,2002-08-28,2017-06-20,Active,0,2002-08-28,Female,...,1,1,1,1,1,1,1,1,1,0


### Overall membership trend

In [95]:
# first look just at overall membership changes year to year

mem_by_year_list = ['mem_2010','mem_2011','mem_2012','mem_2013','mem_2014','mem_2015','mem_2016']

dfmc = pd.DataFrame(dfn[mem_by_year_list])
dfmc.columns = ['2010', '2011', '2012', '2013', '2014', '2015', '2016']
# memcount =[]
memcount = pd.Series(dfmc.sum(axis=0))
print memcount



memcount.plot(kind='line', figsize=(8,4), title="Annual PPP Membership", fontsize=13)


2010    4333
2011    5706
2012    6379
2013    6860
2014    7303
2015    7603
2016    2497
dtype: int64


### Join by Month x Year

In [96]:
dfn_p = pd.pivot_table(dfn, values='mem_no', index='join_year', columns='join_month',\
               aggfunc=len, fill_value=0)

# dfn_p['annual'] = dfn_p.sum(axis=1)

# dfn_p

# dfn_pp = pd.DataFrame(dfn_p)
# dfn_pp.div(dfn_pp.iloc[13])



### could not figure out how to do stacked 100% graph in pandas, did in excel

In [97]:
# dfn_p.to_csv("../../projects/psp/refined_data/join_pivot.csv",\
#              index=True)

### Number of children trend

In [98]:
num_child_cols = ['mem_no','kid_count', 'mem_2010','mem_2011','mem_2012','mem_2013',\
                  'mem_2014','mem_2015','mem_2016']

dfn_num_child = pd.DataFrame(dfn[num_child_cols])

# count of members with child due by year
df_due = pd.DataFrame(dfn_num_child.loc[dfn_num_child['kid_count'] == 0.5])
due_count = df_due.sum(axis=0)
due = pd.Series(due_count)
print due_count

# # count of members with 1 child per year
df_one = pd.DataFrame(dfn_num_child.loc[dfn_num_child['kid_count'] == 1.0])
one_count = df_one.sum(axis=0)
one = pd.Series(one_count)
print one_count

# # count of members with 2 children by year
df_two = pd.DataFrame(dfn_num_child.loc[dfn_num_child['kid_count'] == 2.0])
two_count = df_two.sum(axis=0)
two = pd.Series(two_count)
print two_count

# # count of members with 3 children by year
df_three = pd.DataFrame(dfn_num_child.loc[dfn_num_child['kid_count'] == 3.0])
three_count = df_three.sum(axis=0)
three = pd.Series(three_count)
print three_count

# # count of members with 4 children by year
df_four = pd.DataFrame(dfn_num_child.loc[dfn_num_child['kid_count'] == 4.0])
kfour_count = df_four.sum(axis=0)
four_or_more = pd.Series(kfour_count)
print kfour_count

kid_count    938.5
mem_2010     241.0
mem_2011     360.0
mem_2012     397.0
mem_2013     420.0
mem_2014     508.0
mem_2015     723.0
mem_2016     380.0
dtype: float64
kid_count    6993.0
mem_2010     1688.0
mem_2011     2329.0
mem_2012     2696.0
mem_2013     3021.0
mem_2014     3425.0
mem_2015     3711.0
mem_2016     1176.0
dtype: float64
kid_count    9514.0
mem_2010     1990.0
mem_2011     2536.0
mem_2012     2787.0
mem_2013     2909.0
mem_2014     2849.0
mem_2015     2684.0
mem_2016      771.0
dtype: float64
kid_count    2061.0
mem_2010      296.0
mem_2011      366.0
mem_2012      389.0
mem_2013      410.0
mem_2014      413.0
mem_2015      384.0
mem_2016      129.0
dtype: float64
kid_count    324.0
mem_2010      32.0
mem_2011      37.0
mem_2012      40.0
mem_2013      46.0
mem_2014      46.0
mem_2015      44.0
mem_2016      20.0
dtype: float64


In [99]:
df_kids = pd.concat([due, one, two, three, four_or_more], axis=1)
df_kids.drop(['kid_count'], axis=0, inplace=True)

In [100]:
df_kids.columns=('child_due','1 child','2 children','3 children','4 + children')
df_kids.describe()

,child_due,1 child,2 children,3 children,4 + children
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,432.714286,2578.000000,2360.857143,341.000000,37.857143
std,150.660451,915.694272,766.009014,101.400197,9.388443
min,241.000000,1176.000000,771.000000,129.000000,20.000000
25%,370.000000,2008.500000,2263.000000,331.000000,34.500000
50%,397.000000,2696.000000,2684.000000,384.000000,40.000000
75%,464.000000,3223.000000,2818.000000,399.500000,45.000000
max,723.000000,3711.000000,2909.000000,413.000000,46.000000


In [101]:
### exporting to excel to make 100% stacked chart

# df_kids.to_csv("../../projects/psp/refined_data/kid_count_pivot.csv",\
#              index=True)

### Join v birth trend

In [102]:
# create graph showing difference between when a member joined and the age of child by year

join_v_birth_cols = ['mem_no','join_year','join_v_birth']

dfn_join_v_birth = pd.DataFrame(dfn[join_v_birth_cols])

dfn_join_v_birth.describe()

# calculate months from datetime (mult by 43200)
# 1 year = 518400
# 2 years = 1036800
# 3 years = 1555200
# 5 years = 2592000

# joined while child due
df_due = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] <= 0])
df_due['jvb_range'] = 'joined while due'


# joined while child was under 1 year old
df_one = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] > 0])
df_one = pd.DataFrame(df_one.loc[df_one['join_v_birth'] <= 51840.0 ])
df_one['jvb_range'] = 'joined while child under 1'


# joined while child was 1 - 2 years old
df_two = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] > 518400])
df_two = pd.DataFrame(df_two.loc[df_two['join_v_birth'] <= 1036800.0 ])
df_two['jvb_range'] = 'joined while child 1-2'


# joined while child was 2 - 3 years old
df_three = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] > 1036800.0])
df_three = pd.DataFrame(df_three.loc[df_three['join_v_birth'] <= 1555200.0 ])
df_three['jvb_range'] = 'joined while child 2-3'


# joined while child was 3 - 5 years old
df_five = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] > 1555200.0])
df_five = pd.DataFrame(df_five.loc[df_five['join_v_birth'] <= 2592000.0 ])
df_five['jvb_range'] = 'joined while child 3-5'


# joined while child was 5+ years old
df_older = pd.DataFrame(dfn_join_v_birth.loc[dfn_join_v_birth['join_v_birth'] > 2592000])
df_older['jvb_range'] = 'joined while child 5 or older'


In [103]:
df_one.head()

,mem_no,join_year,join_v_birth,jvb_range
75,00565,2003,21600.0,joined while child under 1
102,00752,2004,31680.0,joined while child under 1
111,00799,2004,31680.0,joined while child under 1
112,00800,2004,30240.0,joined while child under 1
119,00848,2004,44640.0,joined while child under 1


In [104]:
df_jvb = pd.concat([df_due, df_one, df_two, df_five, df_older], axis=0)
# df_kids.drop(['kid_count'], axis=0, inplace=True)
df_jvb.head(20)

,mem_no,join_year,join_v_birth,jvb_range
18,00219,2003,-123840.0,joined while due
23,00238,2003,-1805760.0,joined while due
33,00273,2003,-27360.0,joined while due
34,00278,2003,-266400.0,joined while due
37,00304,2003,-241920.0,joined while due
41,00337,2003,-15840.0,joined while due
47,00383,2003,-4341600.0,joined while due
48,00397,2003,-10080.0,joined while due
50,00409,2003,-172800.0,joined while due
52,00423,2003,-103680.0,joined while due


In [105]:
dfn_jvbpiv = pd.pivot_table(df_jvb, values='mem_no', index='join_year', columns='jvb_range',\
               aggfunc=len, fill_value=0)

In [106]:
### exporting to excel to make 100% stacked chart

dfn_jvbpiv.to_csv("../../projects/psp/refined_data/jvb_pivot.csv",\
             index=True)

### Exp v birth trend

In [107]:
# create graph showing difference between when a memberbership lapsed and the age of youngest child (of 2) by year

exp_v_birth_cols = ['mem_no','exp_year','exp_v_birth']

dfn_exp_v_birth = pd.DataFrame(dfn[exp_v_birth_cols])

dfn_exp_v_birth.describe()

# calculate months from datetime (mult by 43200)
# 1 year = 518400
# 2 years = 1036800
# 3 years = 1555200
# 5 years = 2592000


# exped while child was under 1 year old
df_one_exp = pd.DataFrame(dfn_exp_v_birth.loc[dfn_exp_v_birth['exp_v_birth'] <= 51840.0])
df_one_exp['exp_range'] = 'exped while child under 1'


# exped while child was 1 - 2 years old
df_two_exp = pd.DataFrame(dfn_exp_v_birth.loc[dfn_exp_v_birth['exp_v_birth'] > 518400])
df_two_exp = pd.DataFrame(df_two_exp.loc[df_two_exp['exp_v_birth'] <= 1036800.0 ])
df_two_exp['exp_range'] = 'exped while child 1-2'


# exped while child was 2 - 3 years old
df_three_exp = pd.DataFrame(dfn_exp_v_birth.loc[dfn_exp_v_birth['exp_v_birth'] > 1036800.0])
df_three_exp = pd.DataFrame(df_three_exp.loc[df_three_exp['exp_v_birth'] <= 1555200.0 ])
df_three_exp['exp_range'] = 'exped while child 2-3'


# exped while child was 3 - 5 years old
df_five_exp = pd.DataFrame(dfn_exp_v_birth.loc[dfn_exp_v_birth['exp_v_birth'] > 1555200.0])
df_five_exp = pd.DataFrame(df_five_exp.loc[df_five_exp['exp_v_birth'] <= 2592000.0 ])
df_five_exp['exp_range'] = 'exped while child 3-5'


# exped while child was 5+ years old
df_older_exp = pd.DataFrame(dfn_exp_v_birth.loc[dfn_exp_v_birth['exp_v_birth'] > 2592000])
df_older_exp['exp_range'] = 'exped while child 5 or older'


In [108]:
df_evb = pd.concat([df_one_exp, df_two_exp, df_five_exp, df_older_exp], axis=0)
df_evb.head(20)

,mem_no,exp_year,exp_v_birth,exp_range
755,10065,2010,-21600.0,exped while child under 1
1531,15562,2010,-277920.0,exped while child under 1
1543,15581,2010,-1188000.0,exped while child under 1
2474,17026,2010,-1499040.0,exped while child under 1
3006,17733,2010,-79200.0,exped while child under 1
3258,18056,2013,5760.0,exped while child under 1
3272,18072,2011,-110880.0,exped while child under 1
3432,18278,2011,-201600.0,exped while child under 1
3532,18416,2011,-915840.0,exped while child under 1
3728,18682,2011,-120960.0,exped while child under 1


In [109]:
dfn_evbpiv = pd.pivot_table(df_evb, values='mem_no', index='exp_year', columns='exp_range',\
               aggfunc=len, fill_value=0)

In [110]:
### exporting to excel to make 100% stacked chart

dfn_evbpiv.to_csv("../../projects/psp/refined_data/evb_pivot.csv",\
             index=True)

### Google discovery Trend
I saw that word of mouth from friends/neighbors was overwhelmingly the top way to find out about PSP but Google Search was the #2 (known) means so I'll make a trend for this to see if there are any specific peaks. If there are then maybe there were triggers.

In [111]:
# create graph showing when Google was the way the group was found

# google_cols = ['mem_no','joined','discovered_Found it through a Google search']

google_cols = ['mem_no','joined','join_year','join_month','discovered_Found it through a Google search']

df_ggl = pd.DataFrame(dfn[google_cols])

# break into annual columns
df10 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2010])
df10.rename(columns={'join_month':'joined_2010'}, inplace=True)
df10g = df10.groupby(['joined_2010'])['join_year'].count()

df11 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2011])
df11.rename(columns={'join_month':'joined_2011'}, inplace=True)
df11g = df11.groupby(['joined_2011'])['join_year'].count()

df12 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2012])
df12.rename(columns={'join_month':'joined_2012'}, inplace=True)
df12g = df12.groupby(['joined_2012'])['join_year'].count()

df13 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2013])
df13.rename(columns={'join_month':'joined_2013'}, inplace=True)
df13g = df13.groupby(['joined_2013'])['join_year'].count()

df14 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2014])
df14.rename(columns={'join_month':'joined_2014'}, inplace=True)
df14g = df14.groupby(['joined_2014'])['join_year'].count()

df15 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2015])
df15.rename(columns={'join_month':'joined_2015'}, inplace=True)
df15g = df15.groupby(['joined_2015'])['join_year'].count()

df16 = pd.DataFrame(df_ggl.loc[df_ggl['join_year'] == 2016])
df16.rename(columns={'join_month':'joined_2016'}, inplace=True)
df16g = df16.groupby(['joined_2016'])['join_year'].count()

df_ggl = pd.concat([df10g, df11g, df12g, df13g, df14g, df15g, df16g], axis=1)
df_ggl.columns = ['joined_2010', 'joined_2011', 'joined_2012', 'joined_2013',\
                  'joined_2014', 'joined_2015', 'joined_2016']
df_ggl['month']=('Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec')
df_ggl

,joined_2010,joined_2011,joined_2012,joined_2013,joined_2014,joined_2015,joined_2016,month
1,183,180,194,171,189,205,93.0,Jan
2,156,151,129,144,142,154,17.0,Feb
3,149,193,150,120,164,165,55.0,Mar
4,165,172,149,126,133,157,125.0,Apr
5,169,162,147,164,142,136,148.0,May
6,172,182,156,171,154,179,167.0,June
7,154,192,167,181,201,193,149.0,July
8,175,191,147,172,188,198,26.0,Aug
9,145,173,131,167,156,178,13.0,Sept
10,153,147,131,140,173,164,47.0,Oct


In [112]:
### export to csv for graphing
dfn_p.to_csv("../../projects/psp/refined_data/google.csv",\
             index=True)

### Membership Type Trend

In [113]:
print dfn.mem_type

0        5
1        0
2        0
3        2
4        0
5        1
6        0
7        4
8        0
9        0
10       0
11       1
12       0
13       0
14       0
15       5
16       0
17       0
18       0
19       5
20       0
21       0
22       0
23       5
24       2
25       0
26       0
27       0
28       0
29       0
        ..
14561    0
14562    0
14563    0
14564    0
14565    0
14566    0
14567    0
14568    0
14569    0
14570    0
14571    0
14572    0
14573    0
14574    0
14575    0
14576    0
14577    0
14578    0
14579    0
14580    0
14581    0
14582    0
14583    0
14584    0
14585    0
14586    0
14587    0
14588    0
14589    0
14590    0
Name: mem_type, dtype: int64


In [114]:
mem_type_cols = ['mem_no','mem_type', 'mem_2010','mem_2011','mem_2012','mem_2013',\
                  'mem_2014','mem_2015','mem_2016']

dfn_mem_type = pd.DataFrame(dfn[mem_type_cols])

# count of members with 1 year membership
df_mem1 = pd.DataFrame(dfn_mem_type.loc[dfn_mem_type['mem_type'] == 0])
mem1_count = df_mem1.sum(axis=0)
smem1 = pd.Series(mem1_count)
print mem1_count

# # count of members with 2 year membership
df_mem2 = pd.DataFrame(dfn_mem_type.loc[dfn_mem_type['mem_type'] == 1])
mem2_count = df_mem2.sum(axis=0)
smem2 = pd.Series(mem2_count)
# print due_count

# # count of members with 3 year membership
df_mem3 = pd.DataFrame(dfn_mem_type.loc[dfn_mem_type['mem_type'] == 2])
mem3_count = df_mem3.sum(axis=0)
smem3 = pd.Series(mem3_count)
# print due_count

# # count of members with 5 year membership
df_mem5 = pd.DataFrame(dfn_mem_type.loc[dfn_mem_type['mem_type'] == 3])
mem5_count = df_mem5.sum(axis=0)
smem5 = pd.Series(mem5_count)
# print due_count

# # count of members with lifetime membership
df_memlife = pd.DataFrame(dfn_mem_type.loc[dfn_mem_type['mem_type'] == 5])
memlife_count = df_memlife.sum(axis=0)
smemlife = pd.Series(memlife_count)
# print due_count


mem_type       0
mem_2010    3866
mem_2011    5076
mem_2012    5540
mem_2013    5777
mem_2014    6013
mem_2015    6129
mem_2016    2277
dtype: int64


In [115]:
df_memt = pd.concat([smem1, smem2, smem3, smem5, smemlife], axis=1)
df_memt.drop(['mem_no','mem_type'], axis=0, inplace=True)

In [116]:
df_memt

,0,1,2,3,4
mem_2010,3866.0,318.0,79.0,36.0,23.0
mem_2011,5076.0,446.0,107.0,49.0,23.0
mem_2012,5540.0,596.0,154.0,61.0,23.0
mem_2013,5777.0,786.0,194.0,75.0,23.0
mem_2014,6013.0,947.0,231.0,83.0,23.0
mem_2015,6129.0,1085.0,269.0,92.0,23.0
mem_2016,2277.0,186.0,34.0,0.0,0.0


In [117]:
df_memt.columns=('1yr_mem', '2yr_mem', '3yr_mem','5yr_mem','lifetime')
df_memt.describe()

,1yr_mem,2yr_mem,3yr_mem,5yr_mem,lifetime
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,4954.000000,623.428571,152.571429,56.571429,19.714286
std,1407.585877,331.923614,84.630288,31.606208,8.693183
min,2277.000000,186.000000,34.000000,0.000000,0.000000
25%,4471.000000,382.000000,93.000000,42.500000,23.000000
50%,5540.000000,596.000000,154.000000,61.000000,23.000000
75%,5895.000000,866.500000,212.500000,79.000000,23.000000
max,6129.000000,1085.000000,269.000000,92.000000,23.000000


In [118]:
### exporting to excel to make 100% stacked chart

# df_memt.to_csv("../../projects/psp/refined_data/mem_type_pivot.csv",\
#              index=True)

### Google Trends for PSP

In [124]:
goog = pd.read_csv("../../projects/psp/raw_data/multiTimeline.csv")

In [125]:
goog.head()

,Week,Park Slope Parents: (New York)
0,12/11/11,35
1,12/18/11,37
2,12/25/11,12
3,1/1/12,56
4,1/8/12,46


In [127]:
# Create a trace
trace = go.Scatter(
    x = goog['Week'],
    y = goog['Park Slope Parents: (New York)']
)

data = [trace]

# Plot and embed in ipython notebook!
py.iplot(data, filename='basic-line')